# Smart Resource Selection and Orchestration

## Introduction

In this tutorial, we'll explore how OpenDXA agents can intelligently select and use different resources based on task requirements. Resource selection is a crucial capability for building versatile agents that can handle a variety of tasks efficiently.

You'll learn how to create custom resources, integrate Model Control Protocol (MCP) services, and use specialized agents as resources. We'll demonstrate how an agent automatically selects the most appropriate resource for each specific task, leading to more effective problem-solving.

### Learning Objectives
1. Create custom resources by extending BaseResource
2. Use MCP (Model Control Protocol) resources
3. Create specialized agents as resources
4. Understand how agents select resources based on task requirements
5. Combine multiple resources in a single agent

### Prerequisites
- Python 3.8+
- OpenDXA library
- Basic understanding of agents and resources (Tutorial 1-7)

## Conceptual Background

Resources in OpenDXA provide agents with capabilities to access external services, data sources, or specialized functionalities. They act as interfaces between the agent and various tools or services it might need to accomplish its tasks.

Resource selection is the process by which an agent determines which resource is most appropriate for a given task. This selection happens automatically based on the resource descriptions and the nature of the task. Effective resource selection allows agents to be more versatile and efficient, leveraging specialized tools when needed.

In the OpenDXA architecture, resources are abstracted behind consistent interfaces, allowing agents to interact with them in a uniform way regardless of their underlying implementation. This abstraction enables the creation of modular, extensible agent systems.

In [ ]:
from typing import Any, Dict
from opendxa.common.resource.base_resource import BaseResource, ResourceResponse


class RestaurantOptions(BaseResource):
    """Resource for finding and booking restaurants."""

    async def query(self, request: Dict[str, Any]) -> ResourceResponse:
        # Mock implementation - would call restaurant API
        return ResourceResponse(
            success=True,
            content={
                "restaurants": [
                    {
                        "name": "Park Bistro",
                        "cuisine": "American",
                        "rating": 4.5,
                        "price": "$$",
                        "location": "123 Park St",
                        "available_times": ["1:00 PM", "2:00 PM", "3:00 PM"],
                    },
                    {
                        "name": "Garden Cafe",
                        "cuisine": "Farm-to-table",
                        "rating": 4.7,
                        "price": "$$$",
                        "location": "456 Garden Ave",
                        "available_times": ["2:30 PM", "3:30 PM"],
                    },
                ]
            },
        )

class TransportOptions(BaseResource):
    """Resource for checking transportation options."""

    async def query(self, request: Dict[str, Any]) -> ResourceResponse:
        # Mock implementation - would call transport API
        return ResourceResponse(
            success=True,
            content={
                "options": [
                    {
                        "mode": "bus",
                        "route": "45",
                        "duration": "20 mins",
                        "cost": "$2.50",
                    },
                    {"mode": "rideshare", "duration": "10 mins", "cost": "$15.00"},
                ]
            },
        )

# Initialize resources
restaurant_options = RestaurantOptions(
    name="restaurant_service", description="Provides access to restaurant information and booking"
)
transport_options = TransportOptions(
    name="transport_service", description="Provides access to transportation information"
)

print("✓ Restaurant and transport options are now available")

In [ ]:

from typing import Any, Dict
from opendxa.common.resource.base_resource import BaseResource, ResourceResponse


class RestaurantOptions(BaseResource):
    """Resource for finding and booking restaurants."""

    async def query(self, request: Dict[str, Any]) -> ResourceResponse:
        # Mock implementation - would call restaurant API
        return ResourceResponse(
            success=True,
            content={
                "restaurants": [
                    {
                        "name": "Park Bistro",
                        "cuisine": "American",
                        "rating": 4.5,
                        "price": "$$",
                        "location": "123 Park St",
                        "available_times": ["1:00 PM", "2:00 PM", "3:00 PM"],
                    },
                    {
                        "name": "Garden Cafe",
                        "cuisine": "Farm-to-table",
                        "rating": 4.7,
                        "price": "$$$",
                        "location": "456 Garden Ave",
                        "available_times": ["2:30 PM", "3:30 PM"],
                    },
                ]
            },
        )

class TransportOptions(BaseResource):
    """Resource for checking transportation options."""

    async def query(self, request: Dict[str, Any]) -> ResourceResponse:
        # Mock implementation - would call transport API
        return ResourceResponse(
            success=True,
            content={
                "options": [
                    {
                        "mode": "bus",
                        "route": "45",
                        "duration": "20 mins",
                        "cost": "$2.50",
                    },
                    {"mode": "rideshare", "duration": "10 mins", "cost": "$15.00"},
                ]
            },
        )

# Initialize resources
restaurant_options = RestaurantOptions(
    name="restaurant_service", description="Provides access to restaurant information and booking"
)
transport_options = TransportOptions(
    name="transport_service", description="Provides access to transportation information"
)

print("✓ Restaurant and transport options are now available")

In the code above, we've created two custom resources:

1. **RestaurantResource**: Provides information about restaurants, including names, cuisines, ratings, and available times.
2. **TransportResource**: Provides transportation options with details about mode, duration, and cost.

Both resources extend the BaseResource class and implement the query method, which returns a ResourceResponse containing the relevant information. In a real application, these would likely call external APIs instead of returning mock data.

## Integrating MCP Services as Resources

Next, let's create a Model Control Protocol (MCP) resource for accessing a weather service.

In [ ]:
# Define MCP service parameters
from opendxa.common.resource.mcp.mcp_resource import McpResource, StdioTransportParams

# Initialize weather MCP service
weather = McpResource(
    name="weather_mcp_service",
    transport_params=StdioTransportParams(
        server_script="npx",
        command="npx",
        args=["-y", "@h1deya/mcp-server-weather"],
    ),
)

print("✓ Weather service available")

The McpResource provides a standardized way to interact with external services through the Model Control Protocol. In this case, we're setting up a weather service that can be called to get weather forecasts for specific locations.

The MCP resource uses StdioTransportParams to specify how to communicate with the external service - in this case, by running a Python script that implements the weather service.

For more details about MCP resources and how they work, please refer to the `07_mcp_resource.ipynb` tutorial which covers:
- MCP resource architecture and design
- Setting up MCP services
- Communication protocols and parameters
- Best practices for MCP implementation


## Creating Specialized Agents as Helpful Resources

Now, let's create specialized agents that can themselves be used as resources by our main agent.

In [ ]:
# Create specialized agents
from opendxa.agent.agent import Agent
from opendxa.agent.resource.agent_resource import AgentResource


planner = AgentResource(
    name="planner",
    description="Agent responsible for creating structured plans and coordinating "
    "activities based on available information",
    agent=Agent("planner").with_llm({"model": "openai:gpt-4o-mini", "temperature": 0.7, "max_tokens": 1000}),
)

researcher = AgentResource(
    name="researcher",
    description="Agent focused on gathering, analyzing and synthesizing information "
    "from multiple sources to support decision making",
    agent=Agent("researcher").with_llm({"model": "openai:gpt-4o-mini", "temperature": 0.7, "max_tokens": 1000}),
)

print("✓ Planner and researcher agents created")

In this section, we've created two specialized agents that our main agent can use as resources:

1. **Planner Agent**: Specialized in creating structured plans and coordinating activities.
2. **Researcher Agent**: Focused on gathering and synthesizing information from various sources.

Each agent is configured with its own language model and specific parameters. By wrapping these agents as AgentResources, our main agent can leverage their specialized capabilities when needed.

## Combining Resources in a Main Agent

Now that we have our resources ready, let's create a main agent that combines all of them.

In [ ]:
# Create main agent with all resources
agent = Agent("planning_assistant")
agent.with_llm({"model": "openai:gpt-4o-mini", "temperature": 0.7, "max_tokens": 1000})
agent.with_resources(
    {
        "weather": weather,
        "restaurant_options": restaurant_options,
        "transport_options": transport_options,
        "planner": planner,
        "researcher": researcher,
    }
)

print("✓ Main agent created with all resources attached")

Our main agent now has access to all five resources: the weather MCP service, restaurant and transport custom resources, and the planner and researcher specialized agents. The agent will intelligently select which resource to use based on the task at hand.

## Running My Agent with All the Helpful Resources

Let's test how our agent utilizes resources by processing a user query, and providing it with all these helpful resources.

In [ ]:
query = (
    "I'm in San Francisco and would like to plan a nice out-of-town trip. "
    "Could you help me plan: "
    "1. Plan how to get information about the best places to visit in San Francisco "
    "2. Check the weather for this afternoon "
    "3. Suggest the restaurant to go to "
    "Please provide a detailed plan considering all these factors."
)

print("User Query:")
print("-" * 50)
print(query)
print("-" * 50)

print("\nProcessing query...") 
response = agent.ask(query)

print("\nResult:")
print("+" * 80)
print(response)
print("+" * 80)

In this example, the agent processes a user query that requires multiple resources. Based on the nature of each task, the agent intelligently selects the most appropriate resource:

1. For gathering information about tourist attractions, it uses the researcher agent.
2. For checking weather conditions, it uses the weather MCP service.
3. For restaurant recommendations, it uses the restaurant resource.

The agent then combines the information from each resource to provide a comprehensive plan to the user.

For a complete working example of resource selection and coordination,
please refer to `examples/python/02_core_concepts/08_resource_selection.py`

## Exercise

Now, try extending the example by creating a new resource for hotel bookings, and adding it to the main agent.

In [ ]:
from opendxa.common.resource.base_resource import BaseResource, ResourceResponse

# Exercise: Create a HotelResource class
class HotelOptions(BaseResource):
    """Resource for finding and booking hotels."""
    
    async def query(self, request: Dict[str, Any]) -> ResourceResponse:
        # Your implementation here
        # Return a ResourceResponse with hotel information
        return ResourceResponse(success=True, content={"hotel_name": "..."})

# Initialize the hotel resource
# hotel_options = HotelOptions(...)

# Add it to the main agent
# agent.with_resources({"hotel_options": hotel_options})

# Test with a query that requires hotel information
# result = agent.ask("...")

## Real-World Application: Enterprise Trip Planning Assistant

In enterprise settings, coordinating business travel often involves multiple systems and resources. An OpenDXA agent can simplify this process by integrating with corporate travel systems, expense management, calendar scheduling, and local services.

In [ ]:
# Example of an enterprise trip planning system (conceptual)

class EnterpriseCalendarResource(BaseResource):
    """Connects to corporate calendar systems"""
    async def query(self, request: Dict[str, Any]) -> ResourceResponse:
        # Would connect to MS Exchange, Google Workspace, etc.
        return ResourceResponse(success=True, content={"available_slots": ["..."]}) 

class CorporateTravelPolicyResource(BaseResource):
    """Enforces travel policy constraints"""
    async def query(self, request: Dict[str, Any]) -> ResourceResponse:
        # Would check policy limitations, preferred vendors, etc.
        return ResourceResponse(success=True, content={"policy_compliant": True}) 

class ExpenseSystemResource(BaseResource):
    """Interfaces with expense reporting systems"""
    async def query(self, request: Dict[str, Any]) -> ResourceResponse:
        # Would create expense entries, estimate costs, etc.
        return ResourceResponse(success=True, content={"expense_report_id": "ER12345"}) 

# These resources would be combined in an enterprise travel planning agent
# that handles complex multi-system coordination transparently

## Troubleshooting Tips

- **Resource not being selected** - Check that the resource description clearly indicates its capabilities and purpose
- **MCP service connection issues** - Verify the server script path and ensure it's executable
- **Resource returning unexpected results** - Test the resource independently before integrating with the agent
- **Agent using wrong resource for a task** - Make resource descriptions more specific and distinct from each other

## Summary and Next Steps

In this tutorial, you've learned how to create and combine different types of resources in an OpenDXA agent. We covered creating custom resources, integrating MCP services, and using specialized agents as resources. You've seen how the agent intelligently selects the appropriate resource for each task based on the resource descriptions and task requirements.

In the next tutorial, "Advanced Agent Orchestration", you'll learn how to coordinate multiple agents working together on complex tasks, taking resource selection to the next level with sophisticated workflow patterns.

# References

- [OpenDXA Documentation](https://github.com/aitomatic/opendxa)
- [Resource Classes API Reference](https://github.com/aitomatic/opendxa/blob/main/opendxa/common/resource)
- [MCP Resource Documentation](https://github.com/aitomatic/opendxa/blob/main/opendxa/common/resource/mcp/MCP-REFERENCE.md)
- [Smart Resource Selection Example](https://github.com/aitomatic/opendxa/blob/main/examples/python/02_core_concepts/08_smart_resource_selection.py)